# Analisi - Grafi

In [1]:
import pickle
from shapely import Point, LineString, Polygon, MultiPolygon
import geopandas as gpd
from tqdm import tqdm

from my_paths import *
import Graph_functions

## Analisi Automatica - Documentazione

Tramite la funzione **Graph_functions.auto_analysis_poi()** è possibile eseguire in automatico la ricreazione del grafo stradale con il dizionario dei pesi desiderato. Spiegazione della funzione:
 ```python
(function) def auto_analysis_poi(
    list_gdfs_poi: list[dict],
    custom_weights: dict = None,
    PATH_GEOJSON: str = None,
    PATH_PICKLE: str = None
) -> None
```
- Richiede una struttura a liste di dizionari di geoDataFrame per "poi" del tipo:
    ```python
    gdf_list = [
        {"gdf":  gpd.read_file(PATH_DEL_GEOJSON),
        "tipo": "tipo che si vuole assegnare agli archi che connettono il gdf al grafo",
        "attr": {"nome_attributo": "valore_attributo"} # Opzionale
        }
    ]
    ```
    Un attributo che però **non è opzionale** **se** si vuole eseguire la nuova funzione di **"auto_filter_poi"** è **"priorita"** che deve essere un numero  
    intero da 1 in poi. Minore è il valore e maggiore è la priorità (priorità 1 = massima).
    ```python
    "attr": {"priorita": 1}
- custom_weights: Un dizionario di pesi del tipo:
    ```python
    {"cycleway": 0, "primary": 1, ....}
    ```
    Possibilità di inserire come chiave: "default": quando non viene trovata una "highway" associta al dizionario,  
    verrà utilizzato il valore contenuto in "default".  
    In questo modo se si fornisce un dizionario in cui c'è soltanto la chiave "default", tutte le strade avranno quel  
    valore associato (esempio per creare un grafo di strade senza pesi):
    ```python
    {"default": 1}
    ```
    Se non viene dato nessun dizionario di pesi, verrà utilizzato direttamente il grafo salvato in staging, che contiene i pesi
    standard che abbiamo stabilito, salvati su file json.
- Si possono inserire i vari percorsi di salvataggio dei file pickle (grafo) e geojson(geoDataFrame).
    

    ```

Documentazione per **"auto_filter_poi"**
Tramite un grafo (il pickle) generato da un **Graph_functions.auto_analysis_poi()** è possibile lanciare una versione "ridotta" dei percorsi
in base a un numero fissato di "poi".  
Attributi:
```python
(function) def auto_filter_poi(
    G_percorsi: MultiDiGraph,
    max_pois: int,
    peso_inquinamento_vs_incidenti: float = 0.5,
    peso_fattori_esterni_vs_distribuzione_poi: float = 0.5,
    custom_weights: dict = None,
    PATH_GEOJSON: str = None,
    PATH_PICKLE: str = None
) -> None
```
- **G_percorsi** -> Il file pickle salvato da **Graph_functions.auto_analysis_poi()**
- **max_pois** -> Il numero massimo (circa) di "pois" su cui creare la versione ridotta del grafo originale.
- **peso_inquinamento_vs_incidenti** -> Un numero da 0 -> 1: Più basso è e più sarà orientato verso **incidenti** (priorità su inquinamento = 0 per intenderci), mentre al contrario più è alto e più sarà sbilanciato su **inquinamento** rispetto a incidenti. Di base + 0.5 (non sbilanciato).
- **peso_fattori_esterni_vs_distribuzione_poi** -> Stesso discorso di prima ma per quanto riguarda l'impatto dei fattori visti prima, denominati nel loro insieme come **fattori_esterni**. Per **distribuzione_poi** intendiamo la loro "proporzionale" diminuzione in base alla loro normale distribuzione tra i vari municipi.  
Esempio: Municipio 1 ha 10 pois e Municipio 2 ne ha 20 (tot 30) e noi vogliamo un max di 10 -> vista la proporzione di 1 a 2 ->  
Municipio 1 ottiene circa 3 poi mentro Municipio 2 ne ottiene circa 7. Questo nel caso in cui si mettesse come valore 1 a questo attributo e quindi nel caso in cui i "fattori esterni" non vengano tenuti in considerazione, altrimenti potrebbero andare a cambiare questi numeri in base a quanto peso si è deciso di dargli nel computo e poi di conseguenza a quale dei due fattori si è deciso di dare più importanza tramite il precedente parametro.
- Gli altri parametri sono uguali a **auto_filter_poi()**

## Estensione rete ciclabili

ETL gdf:  
Provo a creare dei percorsi che hanno lo scopo di unire tutti i frammenti di ciclabili esistenti.  
Quindi userò come "poi" tutti i nodi delle "highway"=cycleway

In [53]:
gdf_ciclabili = gpd.read_file(PATH_STRADE_CICLABILI_GEOJSON_CLEAN)
gdf_ciclabili_cycleway = gdf_ciclabili[gdf_ciclabili["highway"] == "cycleway"]
gdf_point_ciclabili = []
for idx, row in gdf_ciclabili_cycleway.iterrows():
    gdf_point_ciclabili.append(Point(row.geometry.coords[0]))
    gdf_point_ciclabili.append(Point(row.geometry.coords[-1]))
gdf_point_ciclabili = gpd.GeoDataFrame(gdf_point_ciclabili, columns=["geometry"], crs=CRS_GRAD)

### Analisi Completo

In [54]:
gdf_list = [
    {"gdf":  gdf_point_ciclabili,
     "tipo": "ciclabile",
     "attr": {"priorita": 1}}
]
Graph_functions.auto_analysis_poi(gdf_list,
                                  PATH_PICKLE=PATH_EXTENDED_CICLABILI_COMPLETO_PICKLE_STAGING,
                                  PATH_GEOJSON=PATH_EXTENDED_CICLABILI_COMPLETO_CLEAN)

# --------------------  Post ETL  --------------------------#
# Rimuoviamo gli archi artificiali (in questo caso non ci sono effettivi poi che abbiamo collegato)
gdf_extended_ciclabili = gpd.read_file(PATH_EXTENDED_CICLABILI_COMPLETO_CLEAN)
gdf_extended_ciclabili = gdf_extended_ciclabili[gdf_extended_ciclabili["artificial"]==False]
gdf_extended_ciclabili.reset_index(drop=True).to_file(PATH_EXTENDED_CICLABILI_COMPLETO_CLEAN, driver="GeoJSON")

### Analisi con filtri

Versione Incidenti

In [ ]:
with open(PATH_EXTENDED_CICLABILI_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_extended_ciclabili = pickle.load(file)

n_pois = len(gdf_point_ciclabili) / 100 * 30    # 30% del totale
Graph_functions.auto_filter_poi(G_percorsi = G_extended_ciclabili,
                                max_pois= n_pois,
                                peso_inquinamento_vs_incidenti = 0,
                                peso_fattori_esterni_vs_distribuzione_poi = 1,
                                PATH_GEOJSON = PATH_EXTENDED_CICLABILI_INCIDENTI_CLEAN)

# --------------------  Post ETL  --------------------------#
# Rimuoviamo gli archi artificiali (in questo caso non ci sono effettivi poi che abbiamo collegato)
gdf_extended_ciclabili = gpd.read_file(PATH_EXTENDED_CICLABILI_INCIDENTI_CLEAN)
gdf_extended_ciclabili = gdf_extended_ciclabili[gdf_extended_ciclabili["artificial"]==False]
gdf_extended_ciclabili.reset_index(drop=True).to_file(PATH_EXTENDED_CICLABILI_INCIDENTI_CLEAN, driver="GeoJSON")

Versione Inquinamento

In [ ]:
with open(PATH_EXTENDED_CICLABILI_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_extended_ciclabili = pickle.load(file)

n_pois = len(gdf_point_ciclabili) / 100 * 30    # 30% del totale
Graph_functions.auto_filter_poi(G_percorsi = G_extended_ciclabili,
                                max_pois= n_pois,
                                peso_inquinamento_vs_incidenti = 1,
                                peso_fattori_esterni_vs_distribuzione_poi = 1,
                                PATH_GEOJSON = PATH_EXTENDED_CICLABILI_INQUINAMENTO_CLEAN)

# --------------------  Post ETL  --------------------------#
# Rimuoviamo gli archi artificiali (in questo caso non ci sono effettivi poi che abbiamo collegato)
gdf_extended_ciclabili = gpd.read_file(PATH_EXTENDED_CICLABILI_INQUINAMENTO_CLEAN)
gdf_extended_ciclabili = gdf_extended_ciclabili[gdf_extended_ciclabili["artificial"]==False]
gdf_extended_ciclabili.reset_index(drop=True).to_file(PATH_EXTENDED_CICLABILI_INQUINAMENTO_CLEAN, driver="GeoJSON")

### Unione dei 3 GeoDataFrame

Unione Percorsi generati

In [57]:
gdf_completo = gpd.read_file(PATH_EXTENDED_CICLABILI_COMPLETO_CLEAN)
gdf_filtrato_incidenti = gpd.read_file(PATH_EXTENDED_CICLABILI_INCIDENTI_CLEAN)
gdf_filtrato_inquinamento = gpd.read_file(PATH_EXTENDED_CICLABILI_INQUINAMENTO_CLEAN)

gdf_completo["origine"] = "completo"
gdf_filtrato_incidenti["origine"] = "solo_incidenti"
gdf_filtrato_inquinamento["origine"] = "solo_inquinamento"
gdf_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_completo,
        gdf_filtrato_incidenti,
        gdf_filtrato_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [58]:
gdf_unito.to_file(PATH_EXTENDED_CICLABILI_UNITI_CLEAN)

Formalmente non ci sono punti di interesse da estrarre dai nodi

## Sport e tempo libero

Utilizzeremo i seguenti dati:
1. **Parchi**
2. **Impianti sportivi**
3. **Aree gioco**  

Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

### Analisi completa

In [31]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_PARCHI_CLEAN),
     "tipo": "parchi",
     "attr": {"priorita": 1}},
     {"gdf":  gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN),
     "tipo": "impianti_sportivi",
     "attr": {"priorita": 2}},
     {"gdf":  gpd.read_file(PATH_AREE_GIOCO_CLEAN),
     "tipo": "aree_gioco",
     "attr": {"priorita": 3}} 
]
Graph_functions.auto_analysis_poi(gdf_list,
                                  PATH_PICKLE=PATH_SPORT_TEMPO_LIBERO_COMPLETO_PICKLE_STAGING,
                                  PATH_GEOJSON=PATH_SPORT_TEMPO_LIBERO_COMPLETO_CLEAN)

### Analisi filtrata

Incidenti

In [32]:
with open(PATH_SPORT_TEMPO_LIBERO_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_sport = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_sport,
                                max_pois=400,
                                peso_inquinamento_vs_incidenti=0,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_PICKLE=PATH_SPORT_TEMPO_LIBERO_INCIDENTI_PICKLE_STAGING,
                                PATH_GEOJSON=PATH_SPORT_TEMPO_LIBERO_INCIDENTI_CLEAN)

Inquinamento

In [33]:
with open(PATH_SPORT_TEMPO_LIBERO_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_sport = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_sport,
                                max_pois=400,
                                peso_inquinamento_vs_incidenti=1,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_PICKLE=PATH_SPORT_TEMPO_LIBERO_INQUINAMENTO_PICKLE_STAGING,
                                PATH_GEOJSON=PATH_SPORT_TEMPO_LIBERO_INQUINAMENTO_CLEAN)

### Unione 3 GeoDataFrame

Unione Percorsi generati

In [34]:
gdf_completo = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_COMPLETO_CLEAN)
gdf_filtrato_incidenti = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_INCIDENTI_CLEAN)
gdf_filtrato_inquinamento = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_INQUINAMENTO_CLEAN)

gdf_completo["origine"] = "completo"
gdf_filtrato_incidenti["origine"] = "solo_incidenti"
gdf_filtrato_inquinamento["origine"] = "solo_inquinamento"
gdf_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_completo,
        gdf_filtrato_incidenti,
        gdf_filtrato_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [35]:
gdf_unito.to_file(PATH_SPORT_TEMPO_LIBERO_UNITI_CLEAN)

Unione POI attraverso i Nodi dei grafi

In [67]:
with open(PATH_SPORT_TEMPO_LIBERO_COMPLETO_PICKLE_STAGING, "rb") as file:
    gdf_nodes_completo = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))
with open(PATH_SPORT_TEMPO_LIBERO_INQUINAMENTO_PICKLE_STAGING, "rb") as file:
    gdf_nodes_inquinamento = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))
with open(PATH_SPORT_TEMPO_LIBERO_INCIDENTI_PICKLE_STAGING, "rb") as file:
    gdf_nodes_incidenti = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))

gdf_nodes_completo["origine"] = "completo"
gdf_nodes_inquinamento["origine"] = "solo_inquinamento"
gdf_nodes_incidenti["origine"] = "solo_incidenti"

gdf_nodes_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_nodes_completo,
        gdf_nodes_incidenti,
        gdf_nodes_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)


In [68]:
gdf_nodes_unito.to_file(PATH_SPORT_TEMPO_LIBERO_POI_UNITI_CLEAN, driver="GeoJSON")

## Istruzione

Utilizzeremo i seguenti dati:
1. **Biblioteche**
2. **Scuole**
Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

### Analisi Completa

In [2]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_BIBLIOTECHE_CLEAN),
     "tipo": "biblioteche",
     "attr": {"priorita": 1}},
     {"gdf":  gpd.read_file(PATH_SCUOLE_CLEAN),
     "tipo": "scuole",
     "attr": {"priorita": 2}}
     ]
Graph_functions.auto_analysis_poi(gdf_list,
                                PATH_PICKLE=PATH_ISTRUZIONE_COMPLETO_PICKLE_STAGING,
                                PATH_GEOJSON=PATH_ISTRUZIONE_COMPLETO_CLEAN)

### Analisi Filtrata

Incidenti

In [3]:
with open(PATH_ISTRUZIONE_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_istruzione = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_istruzione,
                                max_pois=100,
                                peso_inquinamento_vs_incidenti=0,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_PICKLE=PATH_ISTRUZIONE_INCIDENTI_PICKLE_STAGING,
                                PATH_GEOJSON=PATH_ISTRUZIONE_INCIDENTI_CLEAN)

Inquinamento

In [4]:
with open(PATH_ISTRUZIONE_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_istruzione = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_istruzione,
                                max_pois=100,
                                peso_inquinamento_vs_incidenti=1,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_PICKLE=PATH_ISTRUZIONE_INQUINAMENTO_PICKLE_STAGING,
                                PATH_GEOJSON=PATH_ISTRUZIONE_INQUINAMENTO_CLEAN)

### Unione 3 GeoDataFrame

Unione Percorsi generati

In [5]:
gdf_completo = gpd.read_file(PATH_ISTRUZIONE_COMPLETO_CLEAN)
gdf_filtrato_incidenti = gpd.read_file(PATH_ISTRUZIONE_INCIDENTI_CLEAN)
gdf_filtrato_inquinamento = gpd.read_file(PATH_ISTRUZIONE_INQUINAMENTO_CLEAN)

gdf_completo["origine"] = "completo"
gdf_filtrato_incidenti["origine"] = "solo_incidenti"
gdf_filtrato_inquinamento["origine"] = "solo_inquinamento"
gdf_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_completo,
        gdf_filtrato_incidenti,
        gdf_filtrato_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [6]:
gdf_unito.to_file(PATH_ISTRUZIONE_UNITI_CLEAN)

Unione POI attraverso i Nodi dei grafi

In [7]:
with open(PATH_ISTRUZIONE_COMPLETO_PICKLE_STAGING, "rb") as file:
    gdf_nodes_completo = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))
with open(PATH_ISTRUZIONE_INQUINAMENTO_PICKLE_STAGING, "rb") as file:
    gdf_nodes_inquinamento = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))
with open(PATH_ISTRUZIONE_INCIDENTI_PICKLE_STAGING, "rb") as file:
    gdf_nodes_incidenti = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))

gdf_nodes_completo["origine"] = "completo"
gdf_nodes_inquinamento["origine"] = "solo_inquinamento"
gdf_nodes_incidenti["origine"] = "solo_incidenti"

gdf_nodes_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_nodes_completo,
        gdf_nodes_incidenti,
        gdf_nodes_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)


In [8]:
gdf_nodes_unito.to_file(PATH_ISTRUZIONE_POI_UNITI_CLEAN, driver="GeoJSON")

## Cultura e spettacolo

Utilizzeremo i seguenti dati: 

1. **Musei**
2. **Cinema**
3. **Teatri**

Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

### Analisi Completa

In [16]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_MUSEI_CLEAN),
     "tipo": "musei",
     "attr": {"priorita": 1}},
     {"gdf":  gpd.read_file(PATH_CINEMA_CLEAN),
     "tipo": "cinema",
     "attr": {"priorita": 2}}, 
     {"gdf":  gpd.read_file(PATH_TEATRI_CLEAN),
     "tipo": "teatri",
     "attr": {"priorita": 3}}
     ]
Graph_functions.auto_analysis_poi(gdf_list,
                                  PATH_PICKLE= PATH_CULTURA_SPETTACOLO_COMPLETO_PICKLE_STAGING,
                                  PATH_GEOJSON=PATH_CULTURA_SPETTACOLO_COMPLETO_CLEAN)

### Analisi Filtrata

Incidenti

In [17]:
with open(PATH_CULTURA_SPETTACOLO_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_cultura_spettacolo = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_cultura_spettacolo,
                                max_pois=100,
                                peso_inquinamento_vs_incidenti=0,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_PICKLE=PATH_CULTURA_SPETTACOLO_INCIDENTI_PICKLE_STAGING,
                                PATH_GEOJSON=PATH_CULTURA_SPETTACOLO_INCIDENTI_CLEAN)

Inquinamento

In [18]:
with open(PATH_CULTURA_SPETTACOLO_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_cultura_spettacolo = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_cultura_spettacolo,
                                max_pois=100,
                                peso_inquinamento_vs_incidenti=1,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_PICKLE=PATH_CULTURA_SPETTACOLO_INQUINAMENTO_PICKLE_STAGING,
                                PATH_GEOJSON=PATH_CULTURA_SPETTACOLO_INQUINAMENTO_CLEAN)

### Unione 3 GeoDataFrame

Unione Percorsi generati

In [19]:
gdf_completo = gpd.read_file(PATH_CULTURA_SPETTACOLO_COMPLETO_CLEAN)
gdf_filtrato_incidenti = gpd.read_file(PATH_CULTURA_SPETTACOLO_INCIDENTI_CLEAN)
gdf_filtrato_inquinamento = gpd.read_file(PATH_CULTURA_SPETTACOLO_INQUINAMENTO_CLEAN)

gdf_completo["origine"] = "completo"
gdf_filtrato_incidenti["origine"] = "solo_incidenti"
gdf_filtrato_inquinamento["origine"] = "solo_inquinamento"
gdf_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_completo,
        gdf_filtrato_incidenti,
        gdf_filtrato_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [20]:
gdf_unito.to_file(PATH_CULTURA_SPETTACOLO_UNITI_CLEAN)

Unione POI attraverso i Nodi dei grafi

In [69]:
with open(PATH_CULTURA_SPETTACOLO_COMPLETO_PICKLE_STAGING, "rb") as file:
    gdf_nodes_completo = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))
with open(PATH_CULTURA_SPETTACOLO_INQUINAMENTO_PICKLE_STAGING, "rb") as file:
    gdf_nodes_inquinamento = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))
with open(PATH_CULTURA_SPETTACOLO_INCIDENTI_PICKLE_STAGING, "rb") as file:
    gdf_nodes_incidenti = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))

gdf_nodes_completo["origine"] = "completo"
gdf_nodes_inquinamento["origine"] = "solo_inquinamento"
gdf_nodes_incidenti["origine"] = "solo_incidenti"

gdf_nodes_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_nodes_completo,
        gdf_nodes_incidenti,
        gdf_nodes_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)


In [70]:
gdf_nodes_unito.to_file(PATH_CULTURA_SPETTACOLO_POI_UNITI_CLEAN, driver="GeoJSON")

## Sanità

Utilizzeremo i seguenti dati: 

1. **Ospedali**
2. **Farmacie**


Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

### Analisi Completa

In [23]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_OSPEDALI_CLEAN),
     "tipo": "ospedali",
     "attr": {"priorita": 1}},
     {"gdf":  gpd.read_file(PATH_FARMACIE_CLEAN),
     "tipo": "farmacie",
     "attr": {"priorita": 2}}
     ]
Graph_functions.auto_analysis_poi(gdf_list,
                                  PATH_PICKLE=PATH_SANITA_COMPLETO_PICKLE_STAGING,
                                  PATH_GEOJSON=PATH_SANITA_COMPLETO_CLEAN)

### Analisi Filtrata

Incidenti

In [24]:
with open(PATH_SANITA_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_sanita = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_sanita,
                                max_pois=300,
                                peso_inquinamento_vs_incidenti=0,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_PICKLE=PATH_SANITA_INCIDENTI_PICKLE_STAGING,
                                PATH_GEOJSON=PATH_SANITA_INCIDENTI_CLEAN)

Inquinamento

In [25]:
with open(PATH_SANITA_COMPLETO_PICKLE_STAGING, "rb") as file:
    G_sanita = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_sanita,
                                max_pois=300,
                                peso_inquinamento_vs_incidenti=1,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_PICKLE=PATH_SANITA_INQUINAMENTO_PICKLE_STAGING,
                                PATH_GEOJSON=PATH_SANITA_INQUINAMENTO_CLEAN)

### Unione 3 GeoDataFrame

Unione Percorsi generati

In [26]:
gdf_completo = gpd.read_file(PATH_SANITA_COMPLETO_CLEAN)
gdf_filtrato_incidenti = gpd.read_file(PATH_SANITA_INCIDENTI_CLEAN)
gdf_filtrato_inquinamento = gpd.read_file(PATH_SANITA_INQUINAMENTO_CLEAN)

gdf_completo["origine"] = "completo"
gdf_filtrato_incidenti["origine"] = "solo_incidenti"
gdf_filtrato_inquinamento["origine"] = "solo_inquinamento"
gdf_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_completo,
        gdf_filtrato_incidenti,
        gdf_filtrato_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [27]:
gdf_unito.to_file(PATH_SANITA_UNITI_CLEAN)

Unione POI attraverso i Nodi dei grafi

In [71]:
with open(PATH_SANITA_COMPLETO_PICKLE_STAGING, "rb") as file:
    gdf_nodes_completo = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))
with open(PATH_SANITA_INQUINAMENTO_PICKLE_STAGING, "rb") as file:
    gdf_nodes_inquinamento = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))
with open(PATH_SANITA_INCIDENTI_PICKLE_STAGING, "rb") as file:
    gdf_nodes_incidenti = Graph_functions.get_gdf_POI_from_graph(pickle.load(file))

gdf_nodes_completo["origine"] = "completo"
gdf_nodes_inquinamento["origine"] = "solo_inquinamento"
gdf_nodes_incidenti["origine"] = "solo_incidenti"

gdf_nodes_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_nodes_completo,
        gdf_nodes_incidenti,
        gdf_nodes_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [72]:
gdf_nodes_unito.to_file(PATH_SANITA_POI_UNITI_CLEAN, driver="GeoJSON")

## Statistiche POI x Municipi

In [47]:
# Carica dati, aggiungendo una colonna "tipo" e tenendo solo questa colonna e geometry
municipi = gpd.read_file(PATH_MUNICIPI_CLEAN)
parchi = gpd.read_file(PATH_PARCHI_CLEAN).assign(tipo="parchi")[["tipo", "geometry"]]
fontane = gpd.read_file(PATH_FONTANE_CLEAN).assign(tipo="fontane")[["tipo", "geometry"]]
impianti = gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN).assign(tipo="impianti_sportivi")[["tipo", "geometry"]]
scuole = gpd.read_file(PATH_SCUOLE_CLEAN).assign(tipo="scuole")[["tipo", "geometry"]]
aree_gioco = gpd.read_file(PATH_AREE_GIOCO_CLEAN).assign(tipo="aree_gioco")[["tipo", "geometry"]]
biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN).assign(tipo="biblioteche")[["tipo", "geometry"]]
stazioni_bikemi = gpd.read_file(PATH_BIKEMI_CLEAN).assign(tipo="stazioni_bikemi")[["tipo", "geometry"]]
case_acqua = gpd.read_file(PATH_CASE_ACQUA_CLEAN).assign(tipo="case_acqua")[["tipo", "geometry"]]
cinema = gpd.read_file(PATH_CINEMA_CLEAN).assign(tipo="cinema")[["tipo", "geometry"]]
farmacie = gpd.read_file(PATH_FARMACIE_CLEAN).assign(tipo="farmacie")[["tipo", "geometry"]]
musei = gpd.read_file(PATH_MUSEI_CLEAN).assign(tipo="musei")[["tipo", "geometry"]]
ospedali = gpd.read_file(PATH_OSPEDALI_CLEAN).assign(tipo="ospedali")[["tipo", "geometry"]]
teatri = gpd.read_file(PATH_TEATRI_CLEAN).assign(tipo="teatri")[["tipo", "geometry"]]
# Anche le ciclabili come "poi" per poter poi filtrare con le analisi
gdf_ciclabili = gpd.read_file(PATH_STRADE_CICLABILI_GEOJSON_CLEAN)
gdf_ciclabili_cycleway = gdf_ciclabili[gdf_ciclabili["highway"] == "cycleway"]
gdf_point_ciclabili = []
for idx, row in gdf_ciclabili_cycleway.iterrows():
    gdf_point_ciclabili.append(Point(row.geometry.coords[0]))
    gdf_point_ciclabili.append(Point(row.geometry.coords[-1]))
ciclabili = gpd.GeoDataFrame(gdf_point_ciclabili, columns=["geometry"], crs=CRS_GRAD).assign(tipo="ciclabile")[["tipo", "geometry"]]

Uniamo le righe dei vari dataframe in uno unico

In [48]:
gdf_pois = gpd.pd.concat([
    parchi, fontane, impianti, scuole, aree_gioco, biblioteche,
    stazioni_bikemi, case_acqua, cinema, farmacie, musei, ospedali, teatri, ciclabili
], ignore_index=True)

Tramite sjoin con predicato "intersect": eseguiamo una left-join con il gdf dei municipi in base
all'intersezione delle geometrie

In [49]:
gdf_pois_con_municipio = gpd.sjoin(
    municipi,
    gdf_pois,
    how="left",
    predicate="intersects"
    ).drop(["index_right"], axis=1) # eliminiamo l'index_right (conserva l'indice del dataframe del gdf di destra ma non ci serve)

Raggruppiamo per Municipio e Tipo (mettiamo anche geometry così ci rimane per il geojson).  

In [50]:
conteggi = gdf_pois_con_municipio.groupby(['MUNICIPIO', 'geometry', 'tipo']).size().unstack(fill_value=0).reset_index()

Aggiungiamo una colonna "point_etichetta" a cui assegniamo il "centroid" del poligono dei vari municipi, in modo  
da utilizzarlo su kepler per aggiungere un layer "poin" a cui assegniamo una "label" (etichetta) così visualizziamo  
a schermo il numero dell'attributo preso in considerazione (esempio, numero di parchi = 34, così si vede direttamente sulla mappa  
oltre a vedersi solo il colore. Può essere carino).

In [51]:
conteggi["point_etichetta"] = conteggi.geometry.apply(lambda x: x.centroid)

Riconvertiamo in geodataframe e salviamo su file geojson

In [52]:
conteggi = gpd.GeoDataFrame(conteggi, geometry="geometry", crs=CRS_GRAD)
conteggi.to_file(PATH_SCORE_X_MUNICIPI, driver="GeoJSON")